<p align="center">
    <a target="_blank" href="https://colab.research.google.com/github/LIDSOL/curso-pln-2026-1/blob/main/3_preprocess.ipynb">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
</p> 

# Preprocesamiento y Tokenización

![](https://2.bp.blogspot.com/-oDvCIkIjwXw/VdWWxfvmq3I/AAAAAAAARUE/r0MrmbNzMz8/s1600/inputoutput.jpg)

- Tecnicas de procesamiento del lenguaje depende de las palabras y las oraciones. Debemos identificar estos elementos para poder procesarlos
- Este paso de identificación de palabras y oraciones se le llama segmentación de texto o **tokenización** (*tokenization*)
- Además de la identificación de unidades aplicaremos transformaciones al texto

## ¿Qué es una palabra?

Aunque la definición de lo que es una palabra puede parecer obvia es tremendamente difícil.

- I'm
- we'd
- I've
- Diego's Bicycle

## Elementos del preprocesamiento

- Limpiado
    - Quitar stopwords
    - Quitar elementos de marcado (HTML, XML)
- Normalización
- Tokenización

### Stopwords

In [ ]:
# Stopwords
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
stopwords.words("spanish")[-15:]

### Normalización

<center><img src="https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fimg1.wikia.nocookie.net%2F__cb20140504152558%2Fspongebob%2Fimages%2Fe%2Fe3%2FThe_spongebob.jpg&f=1&nofb=1&ipt=28368023b54a7c84c9100025981b1042d0f4ca3ceaac53be42094cc1c3794348&ipo=images" height=300 width=300></center>

In [ ]:
import unicodedata

def strip_accents(s: str) -> str:
   return ''.join(
       c for c in unicodedata.normalize('NFD', s)
       if unicodedata.category(c) != 'Mn'
   )

In [ ]:
strip_accents("mamá hoy quería que me oigan en el olímpo")

- https://www.unicode.org/reports/tr44/#GC_Values_Table

> And keep in mind, these manipulations may significantly alter the meaning of the text. Accents, Umlauts etc. are not "decoration".
- [oefe](https://stackoverflow.com/users/49793/oefe) - [source](https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string)

In [ ]:
def normalize(words: list, regex=r"\w+", lang="en") -> list:
    stop_lang = "english" if lang=="en" else "spanish"
    lower_words = list(map(str.lower, words))
    return [
        word
        for word in lower_words
        if word not in stopwords.words(stop_lang)
        and re.match(regex, word)
    ]

## Word-base tokenization

In [ ]:
text = """
¡¡¡Mamá prendele a la grabadora!!!, ¿llamaste a las vecinas? Corre la voz porque, efectivamente, !estoy en la tele! 📺
"""

In [ ]:
text.split()

In [ ]:
import re

In [ ]:
# [a-zA-Z_]
regex = r"\w+"
re.findall(regex, text)

In [ ]:
re.findall(regex, "El valor de PI es 3.14159")

![](http://images.wikia.com/battlebears/images/2/2c/Troll_Problem.jpg)

- Vocabularios gigantescos difíciles de procesar
- Generalmente, entre más grande es el vocabulario más pesado será nuestro modelo

**Ejemplo:**
- Si queremos representaciones vectoriales (*spoiler*) de nuestras palabras obtendríamos vectores distintos para palabras similares
    - niño = `v1(39, 34, 5,...)`
    - niños = `v2(9, 4, 0,...)`
    - niña = `v3(2, 1, 1,...)`
    - ...
- Tendríamos tokens con bajísima frecuencia
    - merequetengue = `vn(0,0,1,...)`

### Una solución: Steaming/Lematización (AKA la vieja confiable)

![](https://i.pinimg.com/736x/77/df/89/77df89e6ff57d332ba4e5d7bff723133--meme.jpg)

In [ ]:
from nltk.corpus import brown
nltk.download('brown')

In [ ]:
from collections import Counter

brown_corpus = [word.lower() for word in brown.words() if re.match(regex, word)]
print(brown_corpus[0])
print("Tokens:", len(brown_corpus))
print("Tipos:", len(Counter(brown_corpus)))

In [ ]:
brown_corpus[:10]

In [ ]:
len(set(brown_corpus))

### Steamming

In [ ]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

In [ ]:
stemmed_brown = [stemmer.stem(word) for word in brown_corpus]

In [ ]:
len(set(stemmed_brown))

### Lematización

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

def get_wordnet_pos(tag):
    if tag.startswith('J'):  
        return 'a'
    elif tag.startswith('V'):  
        return 'v'
    elif tag.startswith('N'):  
        return 'n'
    elif tag.startswith('R'):  
        return 'r'
    else:
        return 'n'

def lemmatize(words: list) -> list:
    result = []
    tagged_tokens = pos_tag(words)
    lemmatizer = WordNetLemmatizer()
    for word, tag in tagged_tokens:
        if word.lower() == 'are' or word.lower() in ['is', 'am']:
            result.append(word)
        else:
            result.append(lemmatizer.lemmatize(word, get_wordnet_pos(tag)))
    
    return result

In [ ]:
lemmatized_brown = lemmatize(brown_corpus)

In [ ]:
lemmatized_brown[:10]

In [ ]:
print("Tipos (word-based):", len(Counter(brown_corpus)))
print("Tipos (Steamming):", len(Counter(stemmed_brown)))
print("Tipos (Lemmatized):", len(Counter(lemmatized_brown)))

### More problems?

<img src="https://uploads.dailydot.com/2019/10/Untitled_Goose_Game_Honk.jpeg?auto=compress%2Cformat&ixlib=php-3.3.0" width="250" height="250">

- Métodos dependientes de las lenguas
- Se pierde información
- Ruled-based

## Subword-tokenization salva el día 🦸🏼‍♀️

![](https://gifdb.com/images/high/super-cow-and-chicken-daxvak1q16quwd9p.webp)

- Segmentación de palabras en unidades más pequeñas (*sub-words*)
- Obtenemos tipos menos variados pero con mayores frecuencias
    - Esto le gusta modelos basados en métodos estadísticos
- Palabras frecuentes no deberían separarse
- Palabras largas y raras debería descomponerse en sub-palabras significativas
- Hay métodos estadisticos que no requieren conocimiento a priori de las lenguas

In [ ]:
text = "Let's do tokenization!"
result = ["Let's", "do", "token", "ization", "!"]
print(f"Objetivo: {text} -> {result}")

### Algoritmos

Existen varios algoritmos para hacer *subword-tokenization* como los que se listan a continuación:

- Byte-Pair Encoding (BPE)
- WordPiece
- Unigram

### BPE

- Segmenmentación iterativa, comienza segmentando en secuencias de caracteres
- Junta los pares más frecuentes (*merge operation*)
- Termina cuando se llega al número de *merge operations* especificado o número de vocabulario deseado (*hyperparams*, depende de la implementación)
- Introducido en el paper: [Neural Machine Translation of Rare Words with Subword Units, (Sennrich et al., 2015)](https://arxiv.org/abs/1508.07909)

In [ ]:
%%HTML
<iframe width="960" height="515" src="https://www.youtube.com/embed/HEikzVL-lZU"></iframe>

#### Ejemplo

In [ ]:
!pip install transformers

In [ ]:
SENTENCE = "Let's do this tokenization to enable hypermodernization on my tokens tokenized 👁️👁️👁️!!!"

In [ ]:
from transformers import BertTokenizer 

bpe_tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")
print(bpe_tokenizer.tokenize(SENTENCE))

In [ ]:
encoded_tokens = bpe_tokenizer(SENTENCE)
print(encoded_tokens["input_ids"])

In [ ]:
bpe_tokenizer.decode(encoded_tokens["input_ids"])

### Objetivo de los subword tokenizers


- Buscamos que modelos de redes neuronales tenga datos mas frecuentes
- Esto ayuda a que en principio "aprendan" mejor
- Reducir el numero de tipos (?)
- Reducir el numero de OOV (?)

#### Vamos a tokenizar 🌈
![](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.pinimg.com%2F736x%2F75%2F28%2Fe7%2F7528e71db75a37f0dcf5be8a54e0523f.jpg&f=1&nofb=1&ipt=d08ba1ed7fa9af9c3692703a667271740c22bb8e8f5b9f5f7acb44715e7d47d8&ipo=images)

In [ ]:
def normalize_sent(sent: list[str]) -> list[str]:
    return [
        word.lower()
        for word in sent
        if re.match(r"\w", word) and word.lower() 
    ]

### Corpus Inglés: Gutenberg

In [ ]:
nltk.download('gutenberg')
nltk.download('punkt_tab')

In [ ]:
from nltk.corpus import gutenberg

In [ ]:
gutenberg.fileids()

In [ ]:
gutenberg_sents = gutenberg.sents()

In [ ]:
len(gutenberg_sents)

In [ ]:
" ".join(gutenberg_sents[0])

In [ ]:
%%time
gutenberg_plain_text = "\n".join([" ".join(normalize_sent(sent)) for sent in gutenberg_sents])

print(gutenberg_plain_text[:100])

In [ ]:
gutenberg_words = gutenberg_plain_text.split()

In [ ]:
gutenberg_words[:10]

In [ ]:
len(gutenberg_words)

In [ ]:
len(gutenberg_plain_text)

In [ ]:
with open("gutenberg_plain.txt", "w") as f:
    f.write(gutenberg_plain_text)

### Tokenizando

In [ ]:
gutenberg_types = Counter(gutenberg_words)
len(gutenberg_types)

In [ ]:
gutenberg_tokenized = bpe_tokenizer.tokenize(gutenberg_plain_text)
gutenberg_tokenized_types = Counter(gutenberg_tokenized)
len(gutenberg_tokenized_types)

In [ ]:
gutenberg_tokenized[:10]

In [ ]:
gutenberg_types.most_common(10)

In [ ]:
gutenberg_tokenized_types.most_common(10)

In [ ]:
gutenberg_lemmatized_types = Counter(lemmatize(gutenberg_words))
len(gutenberg_lemmatized_types)

In [ ]:
gutenberg_lemmatized_types.most_common(10)

### OOV: out of vocabulary

Palabras que se vieron en el entrenamiento pero no estan en el test

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(
    gutenberg_words, test_size=0.3, random_state=42
)
print(len(train_data), len(test_data))

In [ ]:
s_1 = {"a", "b", "c", "d", "e"}
s_2 = {"a", "x", "y", "d"}
print(s_1 - s_2)
print(s_2 - s_1)

In [ ]:
oov_test = set(test_data) - set(train_data)
len(oov_test)

In [ ]:
for word in list(oov_test)[:3]:
    print(f"{word} in train: {word in set(train_data)}")

In [ ]:
train_tokenized, test_tokenized = train_test_split(gutenberg_tokenized, test_size=0.3, random_state=42)
print(len(train_tokenized), len(test_tokenized))

In [ ]:
oov_tokenized_test = set(test_tokenized) - set(train_tokenized)
len(oov_tokenized_test)

## Entrenando nuestro modelo con BPE
![](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmedia1.tenor.com%2Fimages%2Fd565618bb1217a7c435579d9172270d0%2Ftenor.gif%3Fitemid%3D3379322&f=1&nofb=1&ipt=9719714edb643995ce9d978c8bab77f5310204960093070e37e183d5372096d9&ipo=images)

#### https://github.com/rsennrich/subword-nmt

In [ ]:
!pip install subword-nmt

In [ ]:
!ls

In [ ]:
!head gutenberg_plain.txt

In [ ]:
!subword-nmt learn-bpe -s 300 < gutenberg_plain.txt > gutenberg_low.model

In [ ]:
!echo "I need to process this sentence because tokenization can be useful" | subword-nmt apply-bpe -c gutenberg_low.model

In [ ]:
!subword-nmt learn-bpe -s 2500 < gutenberg_plain.txt > gutenberg_high.model

In [ ]:
!echo "I need to process this sentence because tokenization can be useful" | subword-nmt apply-bpe -c gutenberg_high.model

## Ejercicio: Aplicar nuestro modelo a otros corpus: La biblia 📖🇻🇦

In [ ]:
BIBLE_FILE_NAMES = {"spa": "spa-x-bible-reinavaleracontemporanea", "eng": "eng-x-bible-kingjames"}
CORPORA_PATH = ""

In [ ]:
import requests


def get_bible_corpus(lang: str) -> str:
    file_name = BIBLE_FILE_NAMES[lang]
    r = requests.get(
        f"https://raw.githubusercontent.com/ximenina/theturningpoint/main/Detailed/corpora/corpusPBC/{file_name}.txt.clean.txt"
    )
    return r.text


def write_plain_text_corpus(raw_text: str, file_name: str) -> None:
    with open(f"{file_name}.txt", "w") as f:
        f.write(raw_text)

### 1. Aplicalo a la biblia en Inglés

In [ ]:
eng_bible_plain_text = get_bible_corpus("eng")
eng_bible_words = eng_bible_plain_text.lower().replace("\n", " ").split()

eng_bible_types = Counter(eng_bible_words)
len(eng_bible_types)

In [ ]:
eng_bible_lemmas_types = Counter(lemmatize(eng_bible_words))
len(eng_bible_lemmas_types)

In [ ]:
write_plain_text_corpus(eng_bible_plain_text, "eng-bible")

In [ ]:
!subword-nmt apply-bpe -c gutenberg_high.model < eng-bible.txt > eng-bible-tokenized.txt

In [ ]:
with open("eng-bible-tokenized.txt", 'r') as f:
    tokenized_data = f.read()
eng_bible_tokenized = tokenized_data.split()

In [ ]:
eng_bible_tokenized_types = Counter(eng_bible_tokenized)
len(eng_bible_tokenized_types)

In [ ]:
eng_bible_tokenized_types.most_common(30)

### 2. Aplicalo a la biblia en Español

**¿Qué pasa si aplicamos el modelo aprendido con Gutenberg a otras lenguas?**

In [ ]:
spa_bible_plain_text = get_bible_corpus('spa')
spa_bible_words = spa_bible_plain_text.replace("\n", " ").lower().split()

In [ ]:
spa_bible_types = Counter(spa_bible_words)
len(spa_bible_types)

In [ ]:
spa_bible_lemmas_types = Counter(lemmatize(spa_bible_words))
len(spa_bible_lemmas_types)

In [ ]:
write_plain_text_corpus(spa_bible_plain_text, "spa-bible")

In [ ]:
!subword-nmt apply-bpe -c gutenberg_high.model < spa-bible.txt > spa-bible-tokenized.txt

In [ ]:
with open("spa-bible-tokenized.txt", "r") as f:
    tokenized_text = f.read()
spa_bible_tokenized = tokenized_text.split()

In [ ]:
spa_bible_tokenized_types = Counter(spa_bible_tokenized)
len(spa_bible_tokenized_types)

### Type-token Ratio (TTR)

- Una forma de medir la variazión del vocabulario en un corpus
- Este se calcula como $TTR = \frac{len(types)}{len(tokens)}$
- Puede ser útil para monitorear la variación lexica de un texto

In [ ]:
print("Información de la biblia en Inglés")
print("Tokens:", len(eng_bible_words))
print("Types (word-base):", len(eng_bible_types))
print("Types (lemmatized)", len(eng_bible_lemmas_types))
print("Types (BPE):", len(eng_bible_tokenized_types))
print("TTR (word-base):", len(eng_bible_types)/len(eng_bible_words))
print("TTR (BPE):", len(eng_bible_tokenized_types)/len(eng_bible_tokenized))

In [ ]:
print("Bible Spanish Information")
print("Tokens:", len(spa_bible_words))
print("Types (word-base):", len(spa_bible_types))
print("Types (lemmatized)", len(spa_bible_lemmas_types))
print("Types (BPE):", len(spa_bible_tokenized_types))
print("TTR (word-base):", len(spa_bible_types)/len(spa_bible_words))
print("TTR (BPE):", len(spa_bible_tokenized_types)/len(spa_bible_tokenized))

### Pasos para previos a realizar subword tokenization

1. Normalización
2. Pre-tokenización (aka separar por palabras)


## Normalización

<center><img src="https://articles-img.sftcdn.net/t_articles/auto-mapping-folder/sites/3/2023/04/Fotolog-1280x960-1-1024x768.jpg" width=700 height=700></center>

In [ ]:
METROFLOG_SENTENCE = "lEt'$ dó tHis béttëŕ :)"

In [ ]:
normalize(METROFLOG_SENTENCE)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("huggingface-course/albert-tokenizer-without-normalizer")
tokenizer.convert_ids_to_tokens(tokenizer.encode(METROFLOG_SENTENCE))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("albert-large-v2")
tokenizer.convert_ids_to_tokens(tokenizer.encode(METROFLOG_SENTENCE))

#### Y para lenguas de bajos recursos digitales?

- No hay muchos recursos :(
- Pero para el nahuatl esta `pyelotl` :)

#### Normalizando el Nahuatl

In [ ]:
!pip install elotl

In [ ]:
import elotl.corpus
axolotl = elotl.corpus.load("axolotl")

In [ ]:
import elotl.nahuatl.orthography

In [ ]:
# Tres posibles normalizadores: sep, inali, ack
# Sauce: https://pypi.org/project/elotl/

nahuatl_normalizer = elotl.nahuatl.orthography.Normalizer("sep")

In [ ]:
axolotl[1][1]

In [ ]:
nahuatl_normalizer.normalize(axolotl[1][1])

In [ ]:
nahuatl_normalizer.to_phones(axolotl[1][1])

## Entrenando BPE con corpus en Nahuatl

In [ ]:
len(axolotl)

In [ ]:
train_rows_count = len(axolotl) - round(len(axolotl)*.30)

In [ ]:
axolotl_train = axolotl[:train_rows_count]
axolotl_test = axolotl[train_rows_count:]

In [ ]:
axolotl_train[0]

In [ ]:
print("Axolotl train len:", len(axolotl_train))
print("Axolotl test len:", len(axolotl_test))
print("Total:", len(axolotl_test) + len(axolotl_train))

In [ ]:
axolotl_train[:3]

In [ ]:
axolotl_words_train = [word for row in axolotl_train for word in row[1].lower().split()]
len(axolotl_words_train)

In [ ]:
print(axolotl_words_train[:10])

In [ ]:
write_plain_text_corpus(" ".join(axolotl_words_train), "axolotl_plain")

In [ ]:
!subword-nmt learn-bpe -s 500 < axolotl_plain.txt > axolotl.model

In [ ]:
from collections import Counter

axolotl_test_words = [word for row in axolotl_test for word in row[1].lower().split()]
axolotl_test_types = Counter(axolotl_test_words)

In [ ]:
print(axolotl_test_types.most_common(10))

In [ ]:
axolotl_singletons = [singleton for singleton in axolotl_test_types.items() if singleton[1] == 1]

In [ ]:
axolotl_singletons

In [ ]:
len(axolotl_test_types), len(axolotl_singletons)

In [ ]:
write_plain_text_corpus(" ".join(axolotl_test_words), "axolotl_plain_test")

In [ ]:
!subword-nmt apply-bpe -c axolotl.model < axolotl_plain_test.txt > axolotl_tokenized.txt

In [ ]:
with open("axolotl_tokenized.txt") as f:
    axolotl_test_tokenized = f.read().split()

In [ ]:
len(axolotl_test_tokenized)

In [ ]:
print(axolotl_test_tokenized[:10])

In [ ]:
axolotl_test_tokenized_types = Counter(axolotl_test_tokenized)

In [ ]:
axolotl_test_tokenized_types.most_common(20)

In [ ]:
print("Axolotl Information")
print("Tokens:", len(axolotl_test_words))
print("Types (word-base):", len(axolotl_test_types))
print("Types (native BPE):", len(axolotl_test_tokenized_types))
print("TTR (word-base):", len(axolotl_test_types)/len(axolotl_test_words))
print("TTR (BPE):", len(axolotl_test_tokenized_types)/len(axolotl_test_tokenized))